In [1]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
import time
from IPython.display import display, HTML

options = webdriver.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--start-maximized")
options.add_argument("--disable-dev-shm-usage")

In [3]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager(driver_version="141.0.7390.55").install()), options=options)
driver.get("https://orteil.dashnet.org/cookieclicker/")
print(driver.title)
driver.quit()

Cookie Clicker


In [7]:
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager(driver_version="141.0.7390.55").install()),
    options=options
)
driver.get("https://orteil.dashnet.org/cookieclicker/")

driver.implicitly_wait(10)

# Select English
driver.find_element(By.ID, "langSelect-EN").click()
time.sleep(5)

# Locate elements
cookie = driver.find_element(By.ID, "bigCookie")
cookie_count = driver.find_element(By.ID, "cookies")

actions = ActionChains(driver)

for i in range(100):
    actions.click(cookie).perform()
    # Optional: check cookies count every 10 clicks
    if i % 10 == 0:
        print(int(cookie_count.text.split(" ")[0]))
    time.sleep(0.001)

0
10
20
30
40
50
60
70
80
90


In [15]:
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager(driver_version="141.0.7390.55").install()),
    options=options
)
driver.get("https://orteil.dashnet.org/cookieclicker/")

driver.implicitly_wait(10)

# Select English
driver.find_element(By.ID, "langSelect-EN").click()
time.sleep(5)

# Locate main elements
cookie = driver.find_element(By.ID, "bigCookie")
cookie_count = driver.find_element(By.ID, "cookies")

# Build item list (0 = cheapest, higher = more expensive)
# so build it in descending order
items = [driver.find_element(By.ID, f"productPrice{i}") for i in range(3, -1, -1)]
# ↑ change 3 to however many unlocked items you want to check

actions = ActionChains(driver)

for i in range(100):
    actions.click(cookie).perform()

    count_text = cookie_count.text.split(" ")[0]
    count = int(count_text.replace(",", ""))

    # iterate expensive → cheap
    for item in items:
        value_text = item.text.replace(",", "")
        if not value_text:
            continue

        try:
            value = int(value_text)
        except ValueError:
            continue  # skip weird text like "???"

        if value <= count:
            try:
                driver.execute_script("arguments[0].click();", item)
                break  # stop after buying the most expensive affordable item
            except StaleElementReferenceException:
                continue

    time.sleep(0.01)

In [20]:
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager(driver_version="141.0.7390.55").install()),
    options=options
)
driver.get("https://orteil.dashnet.org/cookieclicker/")

driver.implicitly_wait(10)

# Select English
driver.find_element(By.ID, "langSelect-EN").click()
time.sleep(5)

# Locate main elements
cookie = driver.find_element(By.ID, "bigCookie")
cookie_count = driver.find_element(By.ID, "cookies")

# Build item list (0 = cheapest, higher = more expensive)
# so build it in descending order
items = [driver.find_element(By.ID, f"productPrice{i}") for i in range(2, -1, -1)]

actions = ActionChains(driver) 

for i in range(100): 
    actions.click(cookie).perform() 
    count = int(cookie_count.text.split(" ")[0]) 
    for item in items:
        value_text = item.text.replace(",", "")
        if not value_text:  # skip empty strings
            continue
        if int(value_text) <= count:
            upgrade_actions = ActionChains(driver)
            upgrade_actions.move_to_element(item)
            upgrade_actions.click()
            upgrade_actions.perform()

In [2]:
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager(driver_version="141.0.7390.55").install()),
    options=options
)
# Open Cookie Clicker
driver.get("https://orteil.dashnet.org/cookieclicker/")
driver.implicitly_wait(10)

# Select English
driver.find_element(By.ID, "langSelect-EN").click()
time.sleep(5)  # wait for main game to load

# Locate main elements
cookie = driver.find_element(By.ID, "bigCookie")
cookie_count = driver.find_element(By.ID, "cookies")

# Build item list (0 = cheapest, higher = more expensive)
items = [driver.find_element(By.ID, f"productPrice{i}") for i in range(2, -1, -1)]

actions = ActionChains(driver)

# Main loop: click cookie 100 times
for i in range(100):
    # click the big cookie
    actions.click(cookie).perform()

    # get current cookie count
    count_text = re.sub(r"[^\d]", "", cookie_count.text)
    if not count_text:
        continue
    count = int(count_text)

    # collect all affordable items
    affordable_items = []
    for item in items:
        try:
            value_text = item.text.replace(",", "")
            if not value_text:
                continue
            value = int(value_text)
            if value <= count:
                affordable_items.append((value, item))
        except StaleElementReferenceException:
            continue

    # buy the most expensive affordable item
    if affordable_items:
        _, best_item = max(affordable_items, key=lambda x: x[0])
        upgrade_actions = ActionChains(driver)
        upgrade_actions.move_to_element(best_item)
        upgrade_actions.click()
        upgrade_actions.perform()

    # tiny pause to avoid overloading browser
    time.sleep(0.01)